In [9]:
import asyncio
import json
import os
import yaml
from pathlib import Path
from typing import Any, Dict, Optional, Type, TypeVar, Union, get_type_hints
from dataclasses import dataclass, field, MISSING
from watchdog.observers import Observer
from watchdog.events import FileSystemEventHandler
import logging

__version__ = "0.1.0"
__author__ = "AsyncConfig Team"

T = TypeVar('T')

logger = logging.getLogger(__name__)

In [10]:
class ConfigError(Exception):
    """Base exception for configuration errors."""
    pass


class ValidationError(ConfigError):
    """Raised when configuration validation fails."""
    pass


class LoadError(ConfigError):
    """Raised when configuration loading fails."""
    pass


@dataclass
class ConfigSource:
    """Represents a configuration source with priority and reload capabilities."""
    path: Optional[Path] = None
    env_prefix: Optional[str] = None
    data: Optional[Dict[str, Any]] = None
    priority: int = 0
    watch: bool = False

    def __post_init__(self):
        if self.path:
            self.path = Path(self.path)

In [11]:
class ConfigWatcher(FileSystemEventHandler):
    """File system event handler for configuration hot reloading."""

    def __init__(self, config_manager, paths: list[Path]):
        self.config_manager = config_manager
        self.paths = {str(p.resolve()) for p in paths}
        super().__init__()

    def on_modified(self, event):
        if not event.is_directory and event.src_path in self.paths:
            logger.info(f"Configuration file changed: {event.src_path}")
            asyncio.create_task(self.config_manager._reload_config())

In [12]:
class AsyncConfigManager:
    """
    Modern async configuration manager with type safety and hot reloading.

    Features:
    - Async-first design
    - Type-safe configuration classes
    - Environment variable support
    - Hot reloading
    - Multiple source merging
    - Validation with detailed error messages
    """

    def __init__(self):
        self.sources: list[ConfigSource] = []
        self.observers: list[Observer] = []
        self.config_cache: Dict[str, Any] = {}
        self.reload_callbacks: list[callable] = []
        self._lock = asyncio.Lock()

    def add_source(self, source: ConfigSource) -> "AsyncConfigManager":
        """Add a configuration source."""
        self.sources.append(source)
        self.sources.sort(key=lambda x: x.priority, reverse=True)
        return self

    def add_file(self, path: Union[str, Path], priority: int = 0, watch: bool = False) -> "AsyncConfigManager":
        """Add a file-based configuration source."""
        return self.add_source(ConfigSource(path=path, priority=priority, watch=watch))

    def add_env(self, prefix: str, priority: int = 100) -> "AsyncConfigManager":
        """Add environment variable source."""
        return self.add_source(ConfigSource(env_prefix=prefix, priority=priority))

    def add_dict(self, data: Dict[str, Any], priority: int = 50) -> "AsyncConfigManager":
        """Add dictionary-based configuration source."""
        return self.add_source(ConfigSource(data=data, priority=priority))

    async def load_config(self, config_class: Type[T]) -> T:
        """Load and validate configuration into a typed dataclass."""
        async with self._lock:
            config_data = await self._merge_sources()

            try:
                return self._validate_and_convert(config_data, config_class)
            except Exception as e:
                raise ValidationError(f"Failed to validate configuration: {e}")

    async def _merge_sources(self) -> Dict[str, Any]:
        """Merge configuration from all sources based on priority."""
        merged = {}

        for source in reversed(self.sources):
            try:
                data = await self._load_source(source)
                if data:
                    merged.update(data)
            except Exception as e:
                logger.warning(f"Failed to load source {source}: {e}")

        return merged

    async def _load_source(self, source: ConfigSource) -> Optional[Dict[str, Any]]:
        """Load data from a single configuration source."""
        if source.data:
            return source.data.copy()

        if source.path:
            return await self._load_file(source.path)

        if source.env_prefix:
            return self._load_env_vars(source.env_prefix)

        return None

    async def _load_file(self, path: Path) -> Dict[str, Any]:
        """Load configuration from a file."""
        if not path.exists():
            raise LoadError(f"Configuration file not found: {path}")

        try:
            content = await asyncio.to_thread(path.read_text)

            if path.suffix.lower() == '.json':
                return json.loads(content)
            elif path.suffix.lower() in ['.yml', '.yaml']:
                return yaml.safe_load(content) or {}
            else:
                raise LoadError(f"Unsupported file format: {path.suffix}")

        except Exception as e:
            raise LoadError(f"Failed to load {path}: {e}")

    def _load_env_vars(self, prefix: str) -> Dict[str, Any]:
        """Load environment variables with given prefix."""
        env_vars = {}
        prefix = prefix.upper() + '_'

        for key, value in os.environ.items():
            if key.startswith(prefix):
                config_key = key[len(prefix):].lower()
                env_vars[config_key] = self._convert_env_value(value)

        return env_vars

    def _convert_env_value(self, value: str) -> Any:
        """Convert environment variable string to appropriate type."""
        if value.lower() in ('true', 'false'):
            return value.lower() == 'true'

        try:
            if '.' in value:
                return float(value)
            return int(value)
        except ValueError:
            pass

        try:
            return json.loads(value)
        except json.JSONDecodeError:
            pass

        return value

    def _validate_and_convert(self, data: Dict[str, Any], config_class: Type[T]) -> T:
        """Validate and convert data to the specified configuration class."""
        if not hasattr(config_class, '__dataclass_fields__'):
            raise ValidationError(f"{config_class.__name__} must be a dataclass")

        type_hints = get_type_hints(config_class)
        field_values = {}

        for field_name, field_info in config_class.__dataclass_fields__.items():
            if field_name in data:
                field_value = data[field_name]

                if hasattr(field_info.type, '__dataclass_fields__'):
                    if isinstance(field_value, dict):
                        field_value = self._validate_and_convert(field_value, field_info.type)

                field_values[field_name] = field_value
            elif field_info.default is not MISSING:
                field_values[field_name] = field_info.default
            elif field_info.default_factory is not MISSING:
                field_values[field_name] = field_info.default_factory()
            else:
                raise ValidationError(f"Required field '{field_name}' not found in configuration")

        return config_class(**field_values)

    async def start_watching(self):
        """Start watching configuration files for changes."""
        watch_paths = []

        for source in self.sources:
            if source.watch and source.path:
                watch_paths.append(source.path)

        if watch_paths:
            observer = Observer()
            watcher = ConfigWatcher(self, watch_paths)

            for path in watch_paths:
                observer.schedule(watcher, str(path.parent), recursive=False)

            observer.start()
            self.observers.append(observer)
            logger.info(f"Started watching {len(watch_paths)} configuration files")

    async def stop_watching(self):
        """Stop watching configuration files."""
        for observer in self.observers:
            observer.stop()
            observer.join()
        self.observers.clear()

    async def _reload_config(self):
        """Reload configuration from all sources."""
        try:
            self.config_cache.clear()
            for callback in self.reload_callbacks:
                await callback()
            logger.info("Configuration reloaded successfully")
        except Exception as e:
            logger.error(f"Failed to reload configuration: {e}")

    def on_reload(self, callback: callable):
        """Register a callback to be called when configuration is reloaded."""
        self.reload_callbacks.append(callback)

    async def __aenter__(self):
        await self.start_watching()
        return self

    async def __aexit__(self, exc_type, exc_val, exc_tb):
        await self.stop_watching()

In [13]:
async def load_config(config_class: Type[T],
                     config_file: Optional[Union[str, Path]] = None,
                     env_prefix: Optional[str] = None,
                     watch: bool = False) -> T:
    """
    Convenience function to quickly load configuration.

    Args:
        config_class: Dataclass to load configuration into
        config_file: Optional configuration file path
        env_prefix: Optional environment variable prefix
        watch: Whether to watch for file changes

    Returns:
        Configured instance of config_class
    """
    manager = AsyncConfigManager()

    if config_file:
        manager.add_file(config_file, priority=0, watch=watch)

    if env_prefix:
        manager.add_env(env_prefix, priority=100)

    return await manager.load_config(config_class)

In [14]:
@dataclass
class DatabaseConfig:
    """Example database configuration."""
    host: str = "localhost"
    port: int = 5432
    username: str = "admin"
    password: str = ""
    database: str = "myapp"
    ssl_enabled: bool = False
    pool_size: int = 10


@dataclass
class AppConfig:
    """Example application configuration."""
    debug: bool = False
    log_level: str = "INFO"
    secret_key: str = ""
    database: DatabaseConfig = field(default_factory=DatabaseConfig)
    redis_url: str = "redis://localhost:6379"
    max_workers: int = 4


async def demo_simple_config():
    """Demo simple configuration loading."""

    sample_config = {
        "debug": True,
        "log_level": "DEBUG",
        "secret_key": "dev-secret-key",
        "database": {
            "host": "localhost",
            "port": 5432,
            "username": "testuser",
            "password": "testpass",
            "database": "testdb"
        },
        "max_workers": 8
    }

    manager = AsyncConfigManager()
    manager.add_dict(sample_config, priority=0)

    config = await manager.load_config(AppConfig)

    print("=== Simple Configuration Demo ===")
    print(f"Debug mode: {config.debug}")
    print(f"Log level: {config.log_level}")
    print(f"Database host: {config.database.host}")
    print(f"Database port: {config.database.port}")
    print(f"Max workers: {config.max_workers}")

    return config

In [15]:
async def demo_advanced_config():
    """Demo advanced configuration with multiple sources."""

    base_config = {
        "debug": False,
        "log_level": "INFO",
        "secret_key": "production-secret",
        "max_workers": 4
    }

    override_config = {
        "debug": True,
        "log_level": "DEBUG",
        "database": {
            "host": "dev-db.example.com",
            "port": 5433
        }
    }

    env_config = {
        "secret_key": "env-secret-key",
        "redis_url": "redis://prod-redis:6379"
    }

    print("\n=== Advanced Configuration Demo ===")

    manager = AsyncConfigManager()

    manager.add_dict(base_config, priority=0)
    manager.add_dict(override_config, priority=50)
    manager.add_dict(env_config, priority=100)

    config = await manager.load_config(AppConfig)

    print("Configuration sources merged:")
    print(f"Debug mode: {config.debug} (from override)")
    print(f"Log level: {config.log_level} (from override)")
    print(f"Secret key: {config.secret_key} (from env)")
    print(f"Database host: {config.database.host} (from override)")
    print(f"Redis URL: {config.redis_url} (from env)")

    return config


async def demo_validation():
    """Demo configuration validation."""

    print("\n=== Configuration Validation Demo ===")

    valid_config = {
        "debug": True,
        "log_level": "DEBUG",
        "secret_key": "test-key",
        "database": {
            "host": "localhost",
            "port": 5432
        }
    }

    manager = AsyncConfigManager()
    manager.add_dict(valid_config, priority=0)

    try:
        config = await manager.load_config(AppConfig)
        print("✓ Valid configuration loaded successfully")
        print(f"  Database SSL: {config.database.ssl_enabled} (default value)")
        print(f"  Database pool size: {config.database.pool_size} (default value)")
    except ValidationError as e:
        print(f"✗ Validation error: {e}")

    incomplete_config = {
        "debug": True,
        "log_level": "DEBUG"
    }

    manager2 = AsyncConfigManager()
    manager2.add_dict(incomplete_config, priority=0)

    try:
        config2 = await manager2.load_config(AppConfig)
        print("✓ Configuration with defaults loaded successfully")
        print(f"  Secret key: '{config2.secret_key}' (default empty string)")
    except ValidationError as e:
        print(f"✗ Validation error: {e}")

In [8]:
async def run_demos():
    """Run all demonstration functions."""
    try:
        await demo_simple_config()
        await demo_advanced_config()
        await demo_validation()
        print("\n=== All demos completed successfully! ===")
    except Exception as e:
        print(f"Demo error: {e}")
        import traceback
        traceback.print_exc()



await run_demos()

if __name__ == "__main__":
    try:
        loop = asyncio.get_event_loop()
        if loop.is_running():
            print("Running in Jupyter/IPython environment")
            print("Use: await run_demos()")
        else:
            asyncio.run(run_demos())
    except RuntimeError:
        asyncio.run(run_demos())

=== Simple Configuration Demo ===
Debug mode: True
Log level: DEBUG
Database host: localhost
Database port: 5432
Max workers: 8

=== Advanced Configuration Demo ===
Configuration sources merged:
Debug mode: True (from override)
Log level: DEBUG (from override)
Secret key: env-secret-key (from env)
Database host: dev-db.example.com (from override)
Redis URL: redis://prod-redis:6379 (from env)

=== Configuration Validation Demo ===
✓ Valid configuration loaded successfully
  Database SSL: False (default value)
  Database pool size: 10 (default value)
✓ Configuration with defaults loaded successfully
  Secret key: '' (default empty string)

=== All demos completed successfully! ===
Running in Jupyter/IPython environment
Use: await run_demos()
